#### Loading libraries

In [73]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
21,application_1648745669055_0031,pyspark,idle,Link,Link,None,✔


#### Loading Data

In [75]:
raw_dataset = "s3a://bckt-crm-in-prd/row/dataset_filtrado_com_id.parquet/"
recommendations_user = "s3a://bckt-crm-in-prd/recomendations/user/pp_user_recomendations_04_02_2022_16_37_54.csv/"
recommendations_item = "s3a://bckt-crm-in-prd/recomendations/item/*.csv/"
sensitivy_items = "s3a://bckt-crm-in-prd/IN_DATA_ZIP_ENCRYPTED/crm_produtos (2).csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df_raw = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .parquet(raw_dataset)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
df_recommendations_user = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .load(recommendations_user)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
df_recommendations_item = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .load(recommendations_item)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
df_item_sensity = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .load(sensitivy_items)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
df_recommendations_user.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CUSTOMER_ID: string (nullable = true)
 |-- SKU: integer (nullable = true)
 |-- SCORE: double (nullable = true)

In [81]:
df_recommendations_user.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+------+
|         CUSTOMER_ID|   SKU| SCORE|
+--------------------+------+------+
|268jjX1Fv2icJol+N...|112511|0.9489|
|268jjX1Fv2icJol+N...|993660|0.9463|
|268jjX1Fv2icJol+N...|115351| 0.874|
|268jjX1Fv2icJol+N...|993650|0.8352|
|268jjX1Fv2icJol+N...|860490|0.8329|
+--------------------+------+------+
only showing top 5 rows

#### Unique itens in recommendations

In [82]:
df_recommendations_user.select('SKU').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4931

#### Count of recommendations per user

In [83]:
df_recommendations_user.groupby('CUSTOMER_ID').count().show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-----+
|CUSTOMER_ID             |count|
+------------------------+-----+
|Tidy6iL9tYlAeDovSyrarw==|50   |
|X7zoJYAVf7NYf/S3RMpDLQ==|50   |
|Y6VKfd6xj2JQ0s6/QXjsKQ==|50   |
|xeaEJTBEuvhJbuXAjPbG3A==|50   |
|9cfUuqQcK27DDLAg4PlScA==|50   |
|AVtKRGNmtUwSSkJy3BJNTQ==|50   |
|t2s6DKpIEGWdZyFot7j+hw==|50   |
|xdrEay7ln0L8sR8oa+1wAA==|50   |
|kHmLA4iP/X3nOj2DaoPa4g==|50   |
|xGV2wFpKBRuhomgK0Zn0yw==|50   |
+------------------------+-----+
only showing top 10 rows

In [84]:
df_item_sensity.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cod_item: string (nullable = true)
 |-- desc_varejo: string (nullable = true)
 |-- cod_tipo_item: string (nullable = true)
 |-- desc_tipo_item: string (nullable = true)
 |-- cod_grupo: string (nullable = true)
 |-- desc_grupo: string (nullable = true)
 |-- cod_subgrupo: string (nullable = true)
 |-- desc_subgrupo: string (nullable = true)
 |-- cod_categoria: string (nullable = true)
 |-- desc_categoria: string (nullable = true)
 |-- cod_subcategoria: string (nullable = true)
 |-- desc_subcategoria: string (nullable = true)
 |-- cod_marca: string (nullable = true)
 |-- desc_marca: string (nullable = true)
 |-- cod_sub_marca: string (nullable = true)
 |-- desc_sub_marca: string (nullable = true)
 |-- cod_classe_terapeutica: string (nullable = true)
 |-- desc_classe_terapeutica: string (nullable = true)
 |-- cod_sub_tipo: string (nullable = true)
 |-- desc_sub_tipo: string (nullable = true)
 |-- cod_fabricante: integer (nullable = true)
 |-- desc_fabricante: string (nullable = t

In [85]:
df_item_sensity.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-------------+--------------+---------+--------------------+------------+--------------------+-------------+--------------+----------------+-----------------+---------+--------------------+-------------+--------------+----------------------+-----------------------+------------+-------------+--------------+--------------------+------------------------+------------------+---------+-------------+------------------+----------------------+-----------------+--------------+--------+-----------------+-----------------+-------------------+-------------+------------------+------------+---------------+----------+-----------------+
|cod_item|         desc_varejo|cod_tipo_item|desc_tipo_item|cod_grupo|          desc_grupo|cod_subgrupo|       desc_subgrupo|cod_categoria|desc_categoria|cod_subcategoria|desc_subcategoria|cod_marca|          desc_marca|cod_sub_marca|desc_sub_marca|cod_classe_terapeutica|desc_classe_terapeutica|cod_sub_tipo|desc_sub_tipo|cod_fabricante|   

#### Select only non sensitivy items.

In [86]:
from pyspark.sql.functions import col
 
n_sensiveis = df_item_sensity.filter(
    (df_item_sensity.flag_item_sensivel == "N") & ((df_item_sensity.flag_otc == "S") | 
                                                   (df_item_sensity.desc_tipo_item == "HB"))).select(col("cod_item"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
n_sensiveis.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|cod_item|
+--------+
|  994930|
|  629200|
|  515170|
+--------+
only showing top 3 rows

#### Transform data on integer.

In [88]:
n_sensiveis = n_sensiveis.\
        withColumn("cod_item", col("cod_item").cast("integer"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Filter Sensitivy Items on User Recommendations

#### Create Dataframe from join with only sensitivy items.

In [89]:
merge_users = df_recommendations_user.join(n_sensiveis, df_recommendations_user["SKU"] == n_sensiveis["cod_item"])
merge_users.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+------+--------+
|         CUSTOMER_ID|   SKU| SCORE|cod_item|
+--------------------+------+------+--------+
|268jjX1Fv2icJol+N...|112511|0.9489|  112511|
|268jjX1Fv2icJol+N...|993660|0.9463|  993660|
|268jjX1Fv2icJol+N...|115351| 0.874|  115351|
|268jjX1Fv2icJol+N...|993650|0.8352|  993650|
|268jjX1Fv2icJol+N...|832350|0.7487|  832350|
+--------------------+------+------+--------+
only showing top 5 rows

In [90]:
merge_users = merge_users.drop("cod_item")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
print((merge_users.count(), len(merge_users.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(131812009, 3)

#### Unique itens in recommendations

In [118]:
merge_users.select('CUSTOMER_ID').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3846176

### Results validation

In [176]:
ids_validations = ['DCFzrBYQbjQWZqnVozxKsQ==' ]
df_validation = df_raw.filter(df_raw['cpf'].isin(ids_validations))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
df_rec = merge_users

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
df_rec.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CUSTOMER_ID: string (nullable = true)
 |-- SKU: integer (nullable = true)
 |-- SCORE: double (nullable = true)

In [179]:
df_raw.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cpf: string (nullable = true)
 |-- cod_item: long (nullable = true)
 |-- qtd_efetiva: long (nullable = true)
 |-- desc_varejo: string (nullable = true)
 |-- desc_grupo: string (nullable = true)
 |-- flag_item_sensivel: string (nullable = true)
 |-- desc_categoria: string (nullable = true)
 |-- flag_otc: string (nullable = true)
 |-- desc_tipo_item: string (nullable = true)
 |-- customer_id: long (nullable = true)

#### Recommendation Validation Dataset

In [180]:
df_rec_validation = df_rec.filter(df_rec['CUSTOMER_ID'].isin(ids_validations))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
df_rec_validation.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+------+
|         CUSTOMER_ID|   SKU| SCORE|
+--------------------+------+------+
|DCFzrBYQbjQWZqnVo...|483870|1.1574|
|DCFzrBYQbjQWZqnVo...|993910|1.1464|
+--------------------+------+------+
only showing top 2 rows

In [182]:
df_rec_validation = df_rec_validation.join(df_item_sensity, df_rec_validation["SKU"] == df_item_sensity["cod_item"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [183]:
df_rec_validation.groupby('CUSTOMER_ID').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|         CUSTOMER_ID|count|
+--------------------+-----+
|DCFzrBYQbjQWZqnVo...|   43|
+--------------------+-----+

In [184]:
df_rec_validation.groupby('CUSTOMER_ID').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|         CUSTOMER_ID|count|
+--------------------+-----+
|DCFzrBYQbjQWZqnVo...|   43|
+--------------------+-----+

In [185]:
df_rec_validation.select(
    'CUSTOMER_ID','SKU','desc_varejo','desc_grupo','SCORE'
    ).show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------------------+-----------------+------+
|         CUSTOMER_ID|   SKU|         desc_varejo|       desc_grupo| SCORE|
+--------------------+------+--------------------+-----------------+------+
|DCFzrBYQbjQWZqnVo...|483870|SAL DE FRUTA ENO ...|GASTROINTESTINAIS|1.1574|
|DCFzrBYQbjQWZqnVo...|993910|NAPROXENO 550MG 1...|      DOR E FEBRE|1.1464|
+--------------------+------+--------------------+-----------------+------+
only showing top 2 rows

#### Compras

In [186]:
df_validation.filter(df_validation.cpf == 'DCFzrBYQbjQWZqnVozxKsQ==').select(
    'cpf','desc_varejo','desc_grupo','cod_item'
    ).show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+--------------------------------------+---------------------+--------+
|cpf                     |desc_varejo                           |desc_grupo           |cod_item|
+------------------------+--------------------------------------+---------------------+--------+
|DCFzrBYQbjQWZqnVozxKsQ==|OMEPRAZOL 20MG 28CAP NEOQ GEN         |GASTROINTESTINAIS    |707020  |
|DCFzrBYQbjQWZqnVozxKsQ==|LÂM GILLETTE MACH3 SENSIT LEV 4 PAG 3 |MASCULINO            |872280  |
|DCFzrBYQbjQWZqnVozxKsQ==|SAB LIQ PANVEL SEMPBEM AMOR REFIL200ML|SABONETE E ACESSORIOS|118461  |
|DCFzrBYQbjQWZqnVozxKsQ==|DESOD REXONA MEN AER ACTIVE 2X90G+50% |DESODORANTE          |816690  |
|DCFzrBYQbjQWZqnVozxKsQ==|SAB LIQ RFL LUX FLOR DE LÓTUS 200ML   |SABONETE E ACESSORIOS|119848  |
|DCFzrBYQbjQWZqnVozxKsQ==|PROT SOL C&B F30 200ML GTS FAC F30 50G|PRODUTOS PARA O CORPO|119063  |
|DCFzrBYQbjQWZqnVozxKsQ==|LISADOR DIP 1G 10CP                   |DOR E FEBRE          |869680  |
|DCFzrBYQbjQWZqnVozxKsQ==|ESC 

#### Recomendações

In [187]:
df_rec_validation.filter(df_rec_validation.CUSTOMER_ID == 'DCFzrBYQbjQWZqnVozxKsQ==').select(
    'CUSTOMER_ID','SKU','desc_varejo','desc_grupo','SCORE'
    ).show(50, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+------+--------------------------------------+-----------------------+------+
|CUSTOMER_ID             |SKU   |desc_varejo                           |desc_grupo             |SCORE |
+------------------------+------+--------------------------------------+-----------------------+------+
|DCFzrBYQbjQWZqnVozxKsQ==|483870|SAL DE FRUTA ENO 5G 2ENV AV           |GASTROINTESTINAIS      |1.1574|
|DCFzrBYQbjQWZqnVozxKsQ==|993910|NAPROXENO 550MG 10 CP NEOQ GEN C      |DOR E FEBRE            |1.1464|
|DCFzrBYQbjQWZqnVozxKsQ==|670160|PARACETAMOL 500MG 20 CP PRATI GEN     |DOR E FEBRE            |1.1279|
|DCFzrBYQbjQWZqnVozxKsQ==|109532|ENO MENTA 8 PASTILHAS MASTIGAVEIS AV  |GASTROINTESTINAIS      |1.0312|
|DCFzrBYQbjQWZqnVozxKsQ==|591080|HYABAK 0,15% COL10ML                  |CUIDADOS COM OLHOS     |1.0206|
|DCFzrBYQbjQWZqnVozxKsQ==|370126|STILGRIP GRAN 5G AV CH                |GRIPES E RESFRIADOS    |1.0083|
|DCFzrBYQbjQWZqnVozxKsQ==|156550|PARACETAMOL 750MG 20 CP NEOQ GE

#### Unique desc_grupo

In [29]:
df_validation.filter(df_validation.cpf == '23Ov54Y7xJ3blh+6BBw5cA==').select(
    'desc_grupo').distinct().show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+
|desc_grupo                 |
+---------------------------+
|TRATAMENTO FACIAL          |
|PRIMEIROS SOCORROS         |
|GASTROINTESTINAIS          |
|MAOS E PES                 |
|MASCULINO                  |
|SABONETE E ACESSORIOS      |
|INFANTIL                   |
|DESODORANTE                |
|ACESSORIOS P/ CABELO       |
|VITAMINAS E SUPLEMENTOS    |
|ORTOPEDICO                 |
|CARDIO                     |
|ALERGIAS E INFECÇÕES       |
|CABELOS                    |
|ALIMENTO                   |
|COLESTEROL E TRIGLICERÍDEOS|
|PELE E MUCOSAS             |
|DOR E FEBRE                |
|MAQUIAGEM                  |
|PRODUTOS PARA O CORPO      |
|SAÚDE DA MULHER            |
|PSICOTROPICO               |
+---------------------------+

In [30]:
df_rec_validation.filter(df_rec_validation['CUSTOMER_ID'] == '23Ov54Y7xJ3blh+6BBw5cA==').select(
    'desc_grupo').distinct().show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|desc_grupo            |
+----------------------+
|PRIMEIROS SOCORROS    |
|GASTROINTESTINAIS     |
|GRIPES E RESFRIADOS   |
|MAOS E PES            |
|SABONETE E ACESSORIOS |
|INFANTIL              |
|DESODORANTE           |
|FERIMENTOS E CONTUSÕES|
|HIGIENE INTIMA        |
|ALIMENTO              |
|DOR E FEBRE           |
|MAQUIAGEM             |
|HIGIENE BUCAL         |
|PRODUTOS PARA O CORPO |
+----------------------+

#### Compras

In [31]:
df_validation.filter(df_validation.cpf == '0wpra35PuZlCwsFjZIb3BQ==').select(
    'cpf','desc_varejo','desc_grupo','cod_item'
    ).show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+--------------------------------------+-----------------------+--------+
|cpf                     |desc_varejo                           |desc_grupo             |cod_item|
+------------------------+--------------------------------------+-----------------------+--------+
|0wpra35PuZlCwsFjZIb3BQ==|HASTES FLEXIVEIS C/150 PANVEL. 18     |INFANTIL               |856300  |
|0wpra35PuZlCwsFjZIb3BQ==|ACECLOFENACO 100MG 12CP EMS GEN       |DOR E CONTUSÃO         |623430  |
|0wpra35PuZlCwsFjZIb3BQ==|DIPROSPAN 1X1ML                       |ALERGIAS E INFECÇÕES   |604     |
|0wpra35PuZlCwsFjZIb3BQ==|PEPSAMAR 10 CP AV CH                  |GASTROINTESTINAIS      |913020  |
|0wpra35PuZlCwsFjZIb3BQ==|KIT LBP MURUMURU E ROSA SH+CO300ML+LAT|CABELOS                |110770  |
|0wpra35PuZlCwsFjZIb3BQ==|MILHO MOSTARDA E MEL PANVEL LEVE 25G  |ALIMENTO               |115491  |
|0wpra35PuZlCwsFjZIb3BQ==|MASSAGEOL AER 120ML                   |FERIMENTOS E CONTUSÕES |517280  |
|0wpra35Pu

#### Recomendações

In [32]:
df_rec_validation.filter(df_rec_validation.CUSTOMER_ID == '0wpra35PuZlCwsFjZIb3BQ==').select(
    'CUSTOMER_ID','SKU','desc_varejo','desc_grupo','SCORE'
    ).show(50, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+------+--------------------------------------+-----------------------+------+
|CUSTOMER_ID             |SKU   |desc_varejo                           |desc_grupo             |SCORE |
+------------------------+------+--------------------------------------+-----------------------+------+
|0wpra35PuZlCwsFjZIb3BQ==|478740|LORATADINA 10MG 12 CP BIOS GEN        |GRIPES E RESFRIADOS    |1.3107|
|0wpra35PuZlCwsFjZIb3BQ==|118445|STREPSILS LARANJA 16 PASTILHAS        |GRIPES E RESFRIADOS    |1.2761|
|0wpra35PuZlCwsFjZIb3BQ==|113873|APARELHO DEPIL DESC PANVEL ESSENCIAL19|PRODUTOS PARA O CORPO  |1.2275|
|0wpra35PuZlCwsFjZIb3BQ==|468000|LORATAMED 10MG 12CP                   |GRIPES E RESFRIADOS    |1.2112|
|0wpra35PuZlCwsFjZIb3BQ==|156550|PARACETAMOL 750MG 20 CP NEOQ GEN      |DOR E FEBRE            |1.1687|
|0wpra35PuZlCwsFjZIb3BQ==|111547|SORO FISIOLOGICO LIFAR 0,9% 500ML     |FERIMENTOS E CONTUSÕES |1.1675|
|0wpra35PuZlCwsFjZIb3BQ==|114686|CRE DENT COLGATE LUM. WHITE CAR

In [33]:
df_validation.filter(df_validation.cpf == '0wpra35PuZlCwsFjZIb3BQ==').select(
    'desc_grupo').distinct().show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|desc_grupo             |
+-----------------------+
|TRATAMENTO FACIAL      |
|LINHAS ESPECIAIS       |
|PRIMEIROS SOCORROS     |
|GASTROINTESTINAIS      |
|CUIDADOS COM OLHOS     |
|GRIPES E RESFRIADOS    |
|MAOS E PES             |
|SABONETE E ACESSORIOS  |
|INFANTIL               |
|DESODORANTE            |
|ACESSORIOS P/ CABELO   |
|FERIMENTOS E CONTUSÕES |
|VITAMINAS E SUPLEMENTOS|
|DOR E CONTUSÃO         |
|ALERGIAS E INFECÇÕES   |
|CABELOS                |
|ALIMENTO               |
|CONVENIENCIA           |
|PELE E MUCOSAS         |
|DOR E FEBRE            |
|MAQUIAGEM              |
|HIGIENE BUCAL          |
|PRODUTOS PARA O CORPO  |
|SAÚDE DA MULHER        |
+-----------------------+

In [34]:
df_rec_validation.filter(df_rec_validation['CUSTOMER_ID'] == '0wpra35PuZlCwsFjZIb3BQ==').select(
    'desc_grupo').distinct().show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|desc_grupo             |
+-----------------------+
|TRATAMENTO FACIAL      |
|PRIMEIROS SOCORROS     |
|GRIPES E RESFRIADOS    |
|MAOS E PES             |
|SABONETE E ACESSORIOS  |
|INFANTIL               |
|DESODORANTE            |
|FERIMENTOS E CONTUSÕES |
|VITAMINAS E SUPLEMENTOS|
|MAES E FILHOS          |
|HIGIENE INTIMA         |
|CABELOS                |
|ALIMENTO               |
|DOR E FEBRE            |
|HIGIENE BUCAL          |
|PRODUTOS PARA O CORPO  |
+-----------------------+

..

#### Define functions

In [35]:
def get_purchased(id, n=50):
    print("Purchased")
    df_raw.filter(df_raw.cpf == id).select(
        'cpf','desc_varejo','desc_grupo','qtd_efetiva'
        ).show(n, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def get_recommendations(id, n=50):
    print("Recommendations")
    df_temp = df_rec.join(df_item_sensity, df_rec["SKU"] == df_item_sensity["cod_item"])
    df_temp.filter(df_temp.CUSTOMER_ID == id).select(
    'CUSTOMER_ID','SKU','desc_varejo','desc_grupo','SCORE'
    ).show(n, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
get_purchased('DCFzrBYQbjQWZqnVozxKsQ==',50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Purchased
+------------------------+--------------------------------------+---------------------+-----------+
|cpf                     |desc_varejo                           |desc_grupo           |qtd_efetiva|
+------------------------+--------------------------------------+---------------------+-----------+
|DCFzrBYQbjQWZqnVozxKsQ==|OMEPRAZOL 20MG 28CAP NEOQ GEN         |GASTROINTESTINAIS    |1          |
|DCFzrBYQbjQWZqnVozxKsQ==|LÂM GILLETTE MACH3 SENSIT LEV 4 PAG 3 |MASCULINO            |1          |
|DCFzrBYQbjQWZqnVozxKsQ==|SAB LIQ PANVEL SEMPBEM AMOR REFIL200ML|SABONETE E ACESSORIOS|1          |
|DCFzrBYQbjQWZqnVozxKsQ==|DESOD REXONA MEN AER ACTIVE 2X90G+50% |DESODORANTE          |1          |
|DCFzrBYQbjQWZqnVozxKsQ==|SAB LIQ RFL LUX FLOR DE LÓTUS 200ML   |SABONETE E ACESSORIOS|3          |
|DCFzrBYQbjQWZqnVozxKsQ==|PROT SOL C&B F30 200ML GTS FAC F30 50G|PRODUTOS PARA O CORPO|1          |
|DCFzrBYQbjQWZqnVozxKsQ==|LISADOR DIP 1G 10CP                   |DOR E FEBRE          |2  

In [66]:
get_recommendations('DCFzrBYQbjQWZqnVozxKsQ==',50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recommendations
+------------------------+------+--------------------------------------+-----------------------+------+
|CUSTOMER_ID             |SKU   |desc_varejo                           |desc_grupo             |SCORE |
+------------------------+------+--------------------------------------+-----------------------+------+
|DCFzrBYQbjQWZqnVozxKsQ==|483870|SAL DE FRUTA ENO 5G 2ENV AV           |GASTROINTESTINAIS      |1.1574|
|DCFzrBYQbjQWZqnVozxKsQ==|993910|NAPROXENO 550MG 10 CP NEOQ GEN C      |DOR E FEBRE            |1.1464|
|DCFzrBYQbjQWZqnVozxKsQ==|670160|PARACETAMOL 500MG 20 CP PRATI GEN     |DOR E FEBRE            |1.1279|
|DCFzrBYQbjQWZqnVozxKsQ==|109532|ENO MENTA 8 PASTILHAS MASTIGAVEIS AV  |GASTROINTESTINAIS      |1.0312|
|DCFzrBYQbjQWZqnVozxKsQ==|591080|HYABAK 0,15% COL10ML                  |CUIDADOS COM OLHOS     |1.0206|
|DCFzrBYQbjQWZqnVozxKsQ==|370126|STILGRIP GRAN 5G AV CH                |GRIPES E RESFRIADOS    |1.0083|
|DCFzrBYQbjQWZqnVozxKsQ==|156550|PARACETAMOL 750

#### Unique desc_grupo

In [47]:
df_raw.filter(df_raw.cpf == 'B6XHLKOHzCzw78xEbQJoSA==').select(
    'desc_grupo').distinct().show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|desc_grupo            |
+----------------------+
|TRATAMENTO FACIAL     |
|PRIMEIROS SOCORROS    |
|GASTROINTESTINAIS     |
|CUIDADOS COM OLHOS    |
|GRIPES E RESFRIADOS   |
|MASCULINO             |
|SABONETE E ACESSORIOS |
|INFANTIL              |
|DESODORANTE           |
|FERIMENTOS E CONTUSÕES|
|MAES E FILHOS         |
|CARDIO                |
|ALERGIAS E INFECÇÕES  |
|ALIMENTO              |
|COVID                 |
|CONVENIENCIA          |
|PELE E MUCOSAS        |
|DOR E FEBRE           |
|HIGIENE BUCAL         |
|PSICOTROPICO          |
+----------------------+

In [54]:
df_rec_validation.filter(df_rec_validation['CUSTOMER_ID'] == 'B6XHLKOHzCzw78xEbQJoSA==').select(
    'desc_grupo').distinct().show(50, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|desc_grupo            |
+----------------------+
|TRATAMENTO FACIAL     |
|PRIMEIROS SOCORROS    |
|GASTROINTESTINAIS     |
|GRIPES E RESFRIADOS   |
|MAOS E PES            |
|SABONETE E ACESSORIOS |
|DERMOCOSMETICO        |
|INFANTIL              |
|PRODUTOS PARA GESTANTE|
|ALIMENTO              |
|CONVENIENCIA          |
|DOR E FEBRE           |
|MAQUIAGEM             |
|HIGIENE BUCAL         |
|PRODUTOS PARA O CORPO |
+----------------------+

In [64]:
df_rec.groupBy("SKU") \
  .agg(count("SKU").alias("SKU_count")) \
  .sort(desc("SKU_count")) \
  .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+
|   SKU|SKU_count|
+------+---------+
|156550|  1011121|
|402732|   422914|
|111673|   399340|
|853800|   393016|
|921040|   358881|
|115229|   357842|
|483870|   350862|
|468560|   308561|
|107983|   307834|
|624360|   301961|
|111547|   290448|
|940360|   288856|
|119602|   286969|
|111674|   284019|
|818980|   276615|
|468000|   276254|
|112719|   269522|
|454510|   266897|
|112127|   264929|
|117572|   261712|
+------+---------+
only showing top 20 rows

In [58]:
df_rec.groupby('SKU').count().sort(desc("SKU")).show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+
|   SKU| count|
+------+------+
|998830| 15532|
|998090| 41272|
|996210| 53321|
|996120|169555|
|995490| 18029|
|995480|  3641|
|995190|109534|
|994210|    10|
|993980|125309|
|993960|209462|
|993920| 75100|
|993910|172215|
|993660|141443|
|993650|204140|
|993620|224731|
|993510| 72452|
|993260|  4854|
|992330|    42|
|992320| 30815|
|989620| 59806|
|986170|  8793|
|986150| 10702|
|985950| 32258|
|985430| 44540|
|984360|   413|
|984350| 31376|
|983080|  6222|
|982610|    31|
|980110| 72923|
|980100| 57980|
|979110| 11855|
|979090|  8015|
|979080| 19625|
|977040|129777|
|972960|    94|
|969340|151325|
|968790| 24194|
|968780| 79791|
|968450|     2|
|968430|   248|
|968270|222744|
|968020|    54|
|967650|  7696|
|967640| 59847|
|965590|  9260|
|962790| 12677|
|962420| 17829|
|962410| 80172|
|960980| 23058|
|960440| 46408|
|958050|150078|
|957680| 11635|
|957640|  5300|
|957430| 76098|
|957420| 44038|
|957350| 11282|
|957260| 11133|
|957240| 14785|
|957190| 10387|
|956640|

In [39]:
get_purchased('eQBc3AFC7BMi+14MwjvsgQ==')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Purchased
+------------------------+--------------------------------------+----------------------+-----------+
|cpf                     |desc_varejo                           |desc_grupo            |qtd_efetiva|
+------------------------+--------------------------------------+----------------------+-----------+
|eQBc3AFC7BMi+14MwjvsgQ==|SUMAX 100MG 2 CP                      |DOR E FEBRE           |1          |
|eQBc3AFC7BMi+14MwjvsgQ==|FITA DENTAL PANVEL ORAL SYSTEM 130M   |HIGIENE BUCAL         |1          |
|eQBc3AFC7BMi+14MwjvsgQ==|SUMAX 50MG 2 CP                       |DOR E FEBRE           |2          |
|eQBc3AFC7BMi+14MwjvsgQ==|KIT ESC DENT CURAPROX 5460 U.SOFT C/2 |HIGIENE BUCAL         |1          |
|eQBc3AFC7BMi+14MwjvsgQ==|TORAGESIC 10MG SUBLING 10 CP          |DOR E FEBRE           |1          |
|eQBc3AFC7BMi+14MwjvsgQ==|ALGODAO QUADRADO 50 UND PANVEL. 18    |PRIMEIROS SOCORROS    |1          |
|eQBc3AFC7BMi+14MwjvsgQ==|ESTOMAZIL SEM SABOR PO EFERV 100G     |GASTROINTESTINAI

In [40]:
get_recommendations('eQBc3AFC7BMi+14MwjvsgQ==')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recommendations
+------------------------+------+--------------------------------------+----------------------+------+
|CUSTOMER_ID             |SKU   |desc_varejo                           |desc_grupo            |SCORE |
+------------------------+------+--------------------------------------+----------------------+------+
|eQBc3AFC7BMi+14MwjvsgQ==|114870|PROT SOL NIVEA P&B F30 200 GTS F30 100|PRODUTOS PARA O CORPO |1.2212|
|eQBc3AFC7BMi+14MwjvsgQ==|113631|KIT CRE DEN ORAL-B ESC A.ACUCAR C3 70G|HIGIENE BUCAL         |1.2185|
|eQBc3AFC7BMi+14MwjvsgQ==|110937|MASCARA DESCARTAVEL PANVEL 50 UNIDADES|PRIMEIROS SOCORROS    |1.1716|
|eQBc3AFC7BMi+14MwjvsgQ==|156550|PARACETAMOL 750MG 20 CP NEOQ GEN      |DOR E FEBRE           |1.1513|
|eQBc3AFC7BMi+14MwjvsgQ==|113933|LC AUTOBRONZEAD PANVEL GLOW DARK 120G |PRODUTOS PARA O CORPO |1.1479|
|eQBc3AFC7BMi+14MwjvsgQ==|483870|SAL DE FRUTA ENO 5G 2ENV AV           |GASTROINTESTINAIS     |1.1128|
|eQBc3AFC7BMi+14MwjvsgQ==|675290|ABS S.LIVRE ADAPT PLUS S

In [41]:
get_purchased('LWtkxv07ZvIAEmGY7IXAew==')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Purchased
+------------------------+----------------------------------------+---------------------------+-----------+
|cpf                     |desc_varejo                             |desc_grupo                 |qtd_efetiva|
+------------------------+----------------------------------------+---------------------------+-----------+
|LWtkxv07ZvIAEmGY7IXAew==|METREXATO 2,5MG 24 CP C                 |ONCOLÓGICOS                |1          |
|LWtkxv07ZvIAEmGY7IXAew==|PREDNISONA 5MG 20 CP NEOQ GEN           |ALERGIAS E INFECÇÕES       |1          |
|LWtkxv07ZvIAEmGY7IXAew==|ROSUVASTATINA 5MG 30CP EMS GEN          |COLESTEROL E TRIGLICERÍDEOS|1          |
|LWtkxv07ZvIAEmGY7IXAew==|PURAN T4 100 MCG 30 CP C                |SISTEMA ENDOCRINO          |1          |
|LWtkxv07ZvIAEmGY7IXAew==|BENICAR ANLO 40MG/5MG 30 CP REV C       |CARDIO                     |1          |
|LWtkxv07ZvIAEmGY7IXAew==|PENVIR 500MG 21 CP                      |ALERGIAS E INFECÇÕES       |1          |
|LWtkxv07ZvIAEmGY7

In [42]:
get_recommendations('LWtkxv07ZvIAEmGY7IXAew==')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recommendations
+------------------------+------+--------------------------------------+-----------------------+------+
|CUSTOMER_ID             |SKU   |desc_varejo                           |desc_grupo             |SCORE |
+------------------------+------+--------------------------------------+-----------------------+------+
|LWtkxv07ZvIAEmGY7IXAew==|413990|REP CORP LA ROCHE-POSAY CIC B B5 20ML |DERMOCOSMETICO         |1.3733|
|LWtkxv07ZvIAEmGY7IXAew==|117340|DESOD REXONA AERO CLINICAL CLASS 150ML|DESODORANTE            |1.3573|
|LWtkxv07ZvIAEmGY7IXAew==|113553|ADVIL 400MG (LEVE 8 PAGUE 6) 8 CAP AV |DOR E FEBRE            |1.2161|
|LWtkxv07ZvIAEmGY7IXAew==|675300|PROT DIA CAREFREE PROTECAO LV80PG60   |HIGIENE INTIMA         |1.177 |
|LWtkxv07ZvIAEmGY7IXAew==|463380|GEL ANTISSEPTICO MAOS THAPSYA 55G     |PRIMEIROS SOCORROS     |1.14  |
|LWtkxv07ZvIAEmGY7IXAew==|109865|VITAMINA C EFERV 1G PANVEL VITA 10 CP |VITAMINAS E SUPLEMENTOS|1.1222|
|LWtkxv07ZvIAEmGY7IXAew==|385824|BUSCOPAN COMPOS

In [67]:
get_purchased('0cnCi6ZwgZc0xhKp/YIlEw==')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Purchased
+------------------------+----------------------------------------+---------------------------+-----------+
|cpf                     |desc_varejo                             |desc_grupo                 |qtd_efetiva|
+------------------------+----------------------------------------+---------------------------+-----------+
|0cnCi6ZwgZc0xhKp/YIlEw==|DISFOR CAPS 30 CÁPSULAS                 |VITAMINAS E SUPLEMENTOS    |1          |
|0cnCi6ZwgZc0xhKp/YIlEw==|GEL ANTIS GIOVANNA BABY BLUE 500ML      |PRIMEIROS SOCORROS         |1          |
|0cnCi6ZwgZc0xhKp/YIlEw==|COLIRIO MOURA BRASIL 20ML               |CUIDADOS COM OLHOS         |2          |
|0cnCi6ZwgZc0xhKp/YIlEw==|SHAMP PANTENE MICELAR 200ML             |CABELOS                    |1          |
|0cnCi6ZwgZc0xhKp/YIlEw==|ROSUVASTATINA 10MG 30CP REV EMS GEN     |COLESTEROL E TRIGLICERÍDEOS|1          |
|0cnCi6ZwgZc0xhKp/YIlEw==|SALONPAS ADESIVO GRANDE 2UNID AV        |FERIMENTOS E CONTUSÕES     |2          |
|0cnCi6ZwgZc0xhKp/

In [65]:
get_recommendations('0cnCi6ZwgZc0xhKp/YIlEw==')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recommendations
+------------------------+------+--------------------------------------+----------------------+------+
|CUSTOMER_ID             |SKU   |desc_varejo                           |desc_grupo            |SCORE |
+------------------------+------+--------------------------------------+----------------------+------+
|0cnCi6ZwgZc0xhKp/YIlEw==|109385|MASCARA DESCART INFANT PANVEL 10 UNI  |PRIMEIROS SOCORROS    |1.2634|
|0cnCi6ZwgZc0xhKp/YIlEw==|107190|PROT SOL ISDIN FUS WAT COR FPS50 50ML |DERMOCOSMETICO        |1.201 |
|0cnCi6ZwgZc0xhKp/YIlEw==|853280|ESC DENT COLGATE SLIM SOFT BLACK C/3  |HIGIENE BUCAL         |1.1875|
|0cnCi6ZwgZc0xhKp/YIlEw==|109635|MASC DESC ANTIVIRAL 12H PANVEL 20UN   |PRIMEIROS SOCORROS    |1.174 |
|0cnCi6ZwgZc0xhKp/YIlEw==|468560|DORFLEX 36 COMPRIMIDOS                |DOR E FEBRE           |1.1393|
|0cnCi6ZwgZc0xhKp/YIlEw==|108653|AGUA MINERAL DA PEDRA S/GAS 1,5LTS    |ALIMENTO              |1.0951|
|0cnCi6ZwgZc0xhKp/YIlEw==|116058|ESC DENT ULTRASOFT PANVE

#### Write result to AWS bucket.

In [19]:
merge_users.coalesce(1).write.option("header","true").format("csv").mode("overwrite").save("s3://bckt-crm-in-prd/recomendations/user/pp_user_recomendations_04_01_2022_12_08_17.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Filter Sensitivy Items on Item Recommendations

In [21]:
df_recommendations_item.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CUSTOMER_ID: string (nullable = true)
 |-- SKU: integer (nullable = true)
 |-- SCORE: double (nullable = true)

#### Create Dataframe from join with only sensitivy items.

In [22]:
merge_items = df_recommendations_item.join(n_sensiveis, df_recommendations_item["SKU"] == n_sensiveis["cod_item"])
merge_items.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------+
|         CUSTOMER_ID|   SKU|  SCORE|cod_item|
+--------------------+------+-------+--------+
|hjtpV+vIAw3BUmbhn...|329080| 1.5434|  329080|
|hjtpV+vIAw3BUmbhn...|881230| 3.6458|  881230|
|hjtpV+vIAw3BUmbhn...|117772|11.4597|  117772|
|hjtpV+vIAw3BUmbhn...|117770| 8.2158|  117770|
|hjtpV+vIAw3BUmbhn...|115188| 3.0724|  115188|
+--------------------+------+-------+--------+
only showing top 5 rows

In [23]:
merge_items = merge_items.drop("cod_item")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
print((merge_items.count(), len(merge_items.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(592580000, 3)

#### Unique itens in recommendations

In [25]:
merge_items.select('SKU').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14765

#### Write result to AWS bucket.

In [ ]:
merge_items.coalesce(1).write.option("header","true").format("csv").mode("overwrite").save("s3://bckt-crm-in-prd/recomendations/item/pp_user_recomendations_04_01_2022_12_19_27.csv/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…